# `feyntrop` tutorial: A 2-loop 3-point Feynman graph

The first step is to import the `feyntrop` module, assuming `feyntrop.so` and `py_feyntrop.py` are in the working directory, we can run:

In [1]:
from py_feyntrop import *

## The graph
* We specify the Feynman graph via a list of weighted edges and associated masses.
* Each item of the list is given in the format $((u,w),\nu, m^2)$, where $u$ and $w$ are the vertices connected by the edge, $\nu$ is the edge weight, and $m^2$ is the mass squared. 
* Mass *variables* should be written as strings, but actual values can also be inserted at this stage (strings or not, i.e. both '1/2' and 1/2 are fine).
* We use 0-indexing. So, the vertices are numbered $0,1,2,\ldots$.
* External vertices must come first, so $V_\text{ext} = \{0,1,2\}$ label external partcles, and $V_\text{int} = \{3\}$.

The graph is then given as an edge list:

In [2]:
edges = [((0,1), 1, 'mm'), ((1,3), 1, 'mm'), ((3,2), 1, 'mm'), ((2,0), 1, 'mm'), ((0,3), 1, 'mm')]

## Kinematics

* Next we provide values for the momentum configuration.
* We must provide replacement rules for all scalar products $p_i \cdot p_j \, , \, i \leq j \leq |V_\text{ext}|-2$. In this case $|V_\text{ext}| = 3$.
* The rules are given by tuples `(sp[i,j], var_str)`, where `var_str` is some string of variables or a number (in string format or not, as for the masses above). 
* For instance, given a massless 4-point diagram one could write `(sp[0,2], (-s-t)/2)`, where $s = 2p_0 \cdot p_1 \, , \, t = 2 p_1 \cdot p_2$ are Mandelstam variables and momentum conservation dictates that $u = -s-t = 2 p_0 \cdot p_2$.

In this 3-point example we set $p_0^2 = p_1^2 = 0$ already here, and then define the single kinematic variable by $s_{01} = 2 p_0 \cdot p_1$.

In [3]:
replacement_rules = [(sp[0,0], '0'), (sp[1,1], '0'), (sp[0,1], 's01/2')]

* We have two symbolic variables in `graph` and `replacement_rules`, namely `mm` and `s01`, which must be given numerical values.

We choose $m^2 = 0.2$ and $s_{01} = 1$:

In [4]:
phase_space_point = [('mm', 0.2), ('s01', 1)]

## Additional settings
* `D0` is the integer part of the spacetime dimension $D = D_0 - 2\epsilon$.
* We expand up to but not including `eps_order`.
* `Lambda` denotes the deformation parameter $\lambda$.
* `N` is the number of Monte Carlo sampling points.

In [5]:
D0 = 2
eps_order = 5
Lambda = 7.6
N = int(1e7)

## Tropical integration
* `trop_res` is the value of the Feynman integral *without* any prefactor.
* `Itr` is the normalization factor in the tropical measure.

In [6]:
trop_res, Itr = tropical_integration(N, D0, Lambda, eps_order, edges, replacement_rules, phase_space_point)

Prefactor: gamma(2*eps + 3).
Generalized permutahedron property: fulfilled.
(Effective) kinematic regime: Minkowski
Analytic continuation: activated. Lambda = 7.6
Started integrating using 8 threads and N = 1e+07 points.
Finished in 5.70203 seconds = 0.0015839 hours.

-- eps^0: [-46.56  +/- 0.13]  +  i * [ 87.22  +/- 0.12]
-- eps^1: [-274.49 +/- 0.55]  +  i * [111.45  +/- 0.55]
-- eps^2: [-435.48 +/- 1.30]  +  i * [-174.26 +/- 1.33]
-- eps^3: [-192.42 +/- 2.15]  +  i * [-495.02 +/- 2.15]
-- eps^4: [218.91  +/- 2.69]  +  i * [-432.84 +/- 2.67]


* This yields the $\epsilon$-expansion *with* prefactor $\frac{\Gamma(\omega)}{\Gamma(\nu_1) \cdots \Gamma(\nu_E)} = \Gamma(2\epsilon+3)$ included, where $\omega$ is the superficial degree of divergence and $E$ is the number of edges.

In [7]:
eps_expansion(trop_res, edges, D0)

174.4414847*I - 93.12894659 + eps*(-720.8558509 + 544.8448962*I) + eps**2*(-2116.304712 + 497.7351159*I) + eps**3*(-3575.012126 - 676.3612018*I) + eps**4*(-3877.716614 - 2727.941257*I) + O(eps**5)